In [6]:
import sys
import argparse
import os
import pandas as pd
import numpy as np
import numpy
from collections import Counter
import re
from scipy import stats
import math

sys.path.append('/Users/friedman/Desktop/hypermutationProjectFinal/scripts/utilityScripts')
import configuration_util
filePathDict = configuration_util.get_all_files_path_dict()
writeDir = os.path.join(os.getcwd(), 'FIGURE1_PLOTTING_FILES')

import analysis_utils 
import mutationSigUtils 
import maf_analysis_utils
import clonality_analysis_util

#import msi_analysis_utils
import get_gene_and_cohort_list_utils

In [2]:

def summarize_allele_info(observedMaf, contextMaf, pentasToLookAt):

    listOfDicts = []
    for penta in pentasToLookAt:
        pentasPossible = contextMaf[contextMaf['pentaChange'] == penta]
        possiblePentaAlleles = set(pentasPossible['allele'])
        nPossiblePentas = pentasPossible.shape[0]
        if nPossiblePentas > 0:
            realMafPenta = observedMaf[observedMaf['pentaChange'] == penta]
            observedAlleles = set(realMafPenta['allele'])
            neverObservedAlleles = possiblePentaAlleles - observedAlleles

            alleleCounts = Counter(realMafPenta['allele'])

            for allele, count in alleleCounts.items():
                listOfDicts.append({
                    'allele': allele,
                    'count': count,
                    'penta': penta
                })

            for allele in neverObservedAlleles:
                listOfDicts.append({
                    'allele': allele, 'count': 0, 'penta': penta
                })


    df = pd.DataFrame(listOfDicts)
    return df

In [3]:
#poleMafWithPentaContext = pd.read_table(pathPrefix + '/ifs/work/taylorlab/friedman/myAdjustedDataFiles/poleCaseMafWithPentanucleotideContext.maf')
poleMafWithPentaContext = pd.read_table('~/Desktop/offlineFilesForVirus/poleCaseMafWithPentanucleotideContext.maf')


/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (36,39,85) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
poleMafWithPentaContext['pentaChange'] = poleMafWithPentaContext.apply(lambda row: 
                                    mutationSigUtils.create_strand_specific_pentanucleotide_change(row['Ref_Tri.1'], row['Reference_Allele'], row['Tumor_Seq_Allele2'], row['Variant_Type']), axis=1)


**HOTSPOT MUTATIONS**

In [5]:
#hotspotContextSummary = pd.read_table(pathPrefix + '/ifs/work/taylorlab/friedman/myAdjustedDataFiles/hotspotContextSummary.tsv')
hotspotContextSummary = pd.read_table('~/Desktop/offlineFilesForVirus/hotspotContextSummary.tsv')


In [6]:
hotspotContextSummary['pentaChange'] = hotspotContextSummary.apply(lambda row: 
                                    mutationSigUtils.create_strand_specific_pentanucleotide_change(row['Ref_Tri.1'], row['Reference_Allele'], row['Tumor_Seq_Allele2'], row['Variant_Type']), axis=1)
hotspotContextSummary['allele'] = hotspotContextSummary['Hugo_Symbol'] + '_' + hotspotContextSummary['HGVSp_Short']


In [7]:
topTenPolePentas = [x[0] for x in Counter(poleMafWithPentaContext['pentaChange']).most_common(10)]

In [8]:
poleMafHotspots = poleMafWithPentaContext[poleMafWithPentaContext['is-a-hotspot'] == 'Y']

df = summarize_allele_info(poleMafHotspots, hotspotContextSummary, topTenPolePentas)


In [ ]:
displayThresh = 3
df['gene'] = df['allele'].apply(lambda x: x.split('_')[0])
displayThresh = 5
df['displayLabel'] = df.apply(lambda row: row['allele'] if row['count'] >= displayThresh else None, axis=1)

In [ ]:
df.to_csv('/Users/friedman/Desktop/WORK/dataForLocalPlotting/hotspotAlleleCounts.tsv', index=False, sep='\t')

In [ ]:
df[(df['count'] == 0) & (df['gene'] == 'NF1')]['allele']
#print Counter(df[(df['count'] == 0)]['gene'])

**TRUNCATING MUTATIONS**

In [ ]:
def count_number_of_times_allele_is_in_multiplet(maf, allele):
    casesWithAllele = set(maf[maf['allele'] == allele]['Tumor_Sample_Barcode'])
    if len(casesWithAllele) == 0:
        return None
    alleleMaf = maf[maf['Tumor_Sample_Barcode'].isin(casesWithAllele)]
    gene = allele.split('_')[0]
    otherOncogenicGeneAlleles = alleleMaf[(alleleMaf['oncogenic'].notnull()) & (alleleMaf['Hugo_Symbol'] == gene) &(alleleMaf['allele'] != allele)]
    return (1.0*len(set(otherOncogenicGeneAlleles['Tumor_Sample_Barcode'])))/len(casesWithAllele)

In [ ]:
#lambda function that marks whether an allele is part of a multiplet
def is_allele_multiplet(maf, allele, tsb):
    gene = allele.split('_')[0]
    if maf[(maf['allele'] != allele) & (maf['oncogenic'].notnull()) & (maf['Hugo_Symbol'] == gene) & (maf['Tumor_Sample_Barcode'] == tsb)].shape[0] > 0:
        return 1
    else:
        return 0

In [ ]:
truncatingContextSummary = pd.read_table(pathPrefix + '/ifs/work/taylorlab/friedman/myAdjustedDataFiles/truncatingContextSummary.tsv')

In [ ]:
truncatingContextSummary['pentaChange'] = truncatingContextSummary.apply(lambda row: 
                                    mutationSigUtils.create_strand_specific_pentanucleotide_change(row['Ref_Tri.1'], row['Reference_Allele'], row['Tumor_Seq_Allele2'], row['Variant_Type']), axis=1)


In [ ]:
poleMafWithPentaContext['allele'] = poleMafWithPentaContext['Hugo_Symbol'] + '_' + poleMafWithPentaContext['HGVSp_Short']
truncatingContextSummary['allele'] = truncatingContextSummary['Hugo_Symbol'] + '_' + truncatingContextSummary['HGVSp_Short']
poleMafWithPentaContext = maf_analysis_utils.fix_mll_genes(poleMafWithPentaContext)


In [ ]:
poleMafWithPentaContextTrunc = poleMafWithPentaContext[poleMafWithPentaContext['Consequence'] == 'stop_gained']

dfTruncSum = summarize_allele_info(poleMafWithPentaContextTrunc, truncatingContextSummary, topTenPolePentas)

    

In [ ]:
dfTruncSum['gene'] = dfTruncSum['allele'].apply(lambda x: x.split('_')[0])
displayThresh = 5
dfTruncSum['displayLabel'] = dfTruncSum.apply(lambda row: row['gene'] if row['count'] >= displayThresh else None, axis=1)

In [ ]:
dfTruncSum.to_csv('/Users/friedman/Desktop/WORK/dataForLocalPlotting/alleleCounts.tsv', index=False, sep='\t')

In [ ]:
significantAlleles = set(dfTruncSum[dfTruncSum['displayLabel'].notnull()]['allele'])

In [ ]:
#RELATED/UNRELATED genes as defined by DNDS
relatedGenes = {
    'Endometrial Cancer': ["AKT1","APC","ARID1A","BCOR","CCND1","CDKN2A.p16INK4a","CTCF","CTNNB1",         
"ESR1","FBXW7","FGFR2","KRAS","NRAS","PIK3CA",
                           "PIK3R1","PPP2R1A","PTEN","RB1","RRAS2","SOX17","SPOP","TP53"],
    'Colorectal Cancer':["AMER1","APC","ARID1A","ASXL1","ATM","B2M","BRAF","CDKN2A.p16INK4a","CTNNB1","ELF3","EPHA3","ERBB3","FBXW7","JUN","KRAS","NRAS","PIK3CA","PIK3R1","PTEN","RBM10","RNF43","SMAD2","SMAD3","SMAD4","SOX9","TCF7L2","TP53"],
    'Glioma': ["ARID1A","ARID2","ATRX","BCOR","BRAF","CBL","CDKN1B","CDKN2A.p14arf","CDKN2A.p16INK4a","CDKN2C","CIC","DNMT3A","EGFR","FUBP1","H3F3A","IDH1","KRAS","NF1","NOTCH1","PDGFRA","PIK3CA","PIK3R1","PPM1D","PTEN","PTPN11","RB1","SETD2","STAG2","TP53"]
}

In [ ]:
cohortRelatedGenes = set(relatedGenes['Endometrial Cancer']) | set(relatedGenes['Colorectal Cancer'])

In [ ]:
recurrentAlleleThresh = 5
tumorSuppresors = set(['ERRFI1', 'ASXL2', 'PMAIP1', 'ACTG1', 'SUFU', 'FBXO11', 'MEN1', 'FAM58A', 'B2M', 'RB1', 'DUSP22', 'SESN1', 'GPS2', 'RAD51D', 'SMG1', 'CDC73', 'MAP3K1', 'SMARCB1', 'INPP4B', 'PARK2', 'SMAD4', 'CBFB', 'CDH1', 'PPP6C', 'SETDB1', 'SETDB2', 'NF2', 'CDKN2B', 'CDKN2C', 'CDKN2A', 'DDX3X', 'PIK3R1', 'BARD1', 'PDS5B', 'KLF4', 'SPRED1', 'VHL', 'SMAD2', 'PMS1', 'PMS2', 'SETD2', 'GATA3', 'TBL1XR1', 'MUTYH', 'SOCS1', 'FAM175A', 'ROBO1', 'ARID1B', 'ARID1A', 'TCF7L2', 'STK11', 'FOXA1', 'PTEN', 'FAT1', 'FAS', 'CYLD', 'MAX', 'SH2D1A', 'APC', 'NTHL1', 'CTCF', 'KDM5C', 'KMT2C', 'ZFHX3', 'FOXP1', 'PIGA', 'CDKN1B', 'CDKN1A', 'FUBP1', 'MSH2', 'ID3', 'TNFRSF14', 'TRAF3', 'EP400', 'BRIP1', 'ARID4A', 'ARID4B', 'XRCC2', 'DAXX', 'SDHAF2', 'ASXL1', 'AMER1', 'RASA1', 'EGR1', 'MST1', 'SOX17', 'RUNX1', 'PIK3R3', 'NCOR1', 'NF1', 'JAK1', 'PTPRD', 'CHEK2', 'CHEK1', 'SMC1A', 'TMEM127', 'STAG1', 'RAD51', 'TCF3', 'STAG2', 'ARID2', 'RAD50', 'RNF43', 'PARP1', 'BLM', 'CUX1', 'RECQL', 'RAD21', 'PTPN2', 'PTPN1', 'SLX4', 'INHA', 'PAX5', 'IRF1', 'TP53', 'HLA-A', 'IRF8', 'CBL', 'TOP1', 'SHQ1', 'PRDM1', 'NSD1', 'ATXN2', 'CREBBP', 'HDAC4', 'SESN2', 'PPP2R1A', 'EPHA7', 'ATM', 'EPHA3', 'POT1', 'SMAD3', 'MOB3B', 'TBX3', 'POLE', 'ATR', 'FANCD2', 'FH', 'BCORL1', 'SOX9', 'IKZF3', 'TSC1', 'TP63', 'MRE11A', 'SDHC', 'BTG1', 'POLD1', 'CIITA', 'SMC3', 'SAMHD1', 'RTEL1', 'ECT2L', 'PIK3R2', 'CRBN', 'FANCC', 'NBN', 'FANCA', 'HLA-B', 'RECQL4', 'DUSP4', 'ERCC2', 'FBXW7', 'TGFBR2', 'TGFBR1', 'MSH3', 'RBM15', 'TET1', 'TET3', 'SESN3', 'MGA', 'LTB', 'FOXL2', 'SH2B3', 'BCOR', 'HIST1H1D', 'ATRX', 'EP300', 'RAD51C', 'RAD51B', 'HIST1H1B', 'TNFAIP3', 'DICER1', 'ARID5B', 'LATS2', 'FOXO1', 'KEAP1', 'EZH2', 'SP140', 'NKX3-1', 'PBRM1', 'PALB2', 'CIC', 'BRCA1', 'DTX1', 'FLCN', 'SPEN', 'CD58', 'ERCC3', 'ERCC4', 'MSH6', 'BCL11B', 'BMPR1A', 'ERF', 'BRCA2', 'NOTCH2', 'EED', 'MITF', 'ELF3', 'SMARCA4', 'BBC3', 'ANKRD11', 'CEBPA', 'BCL2L11', 'AXIN2', 'AXIN1', 'CDK12', 'ESCO2', 'MLH1', 'SDHB', 'MED12', 'HNF1A', 'RYBP', 'ATP6V1B2', 'DNMT3B', 'KMT2B', 'KMT2A', 'DNMT3A', 'NFKBIA', 'TRAF5', 'KMT2D', 'SPOP', 'RBM10', 'P2RY8', 'TP53BP1', 'TSC2', 'KDM6A', 'EPCAM', 'PHOX2B', 'NPM1', 'BCL10', 'LATS1', 'HOXB13', 'ARID3A', 'PTPRT', 'PTPRS', 'INPPL1', 'NOTCH4', 'TET2', 'NOTCH1', 'CASP8', 'NOTCH3', 'GRIN2A', 'MAP2K4', 'WT1', 'BACH2', 'SDHA', 'BAP1', 'PTCH1', 'SDHD'])
poleMafTruncatingTSG = poleMafWithPentaContextTrunc[poleMafWithPentaContextTrunc['Hugo_Symbol'].isin(tumorSuppresors)]

recurrentTruncatingAlleles = [allele for allele, count in Counter(poleMafWithPentaContextTrunc['allele']).items() if count > recurrentAlleleThresh]
notRecurrentTruncatingAlleles = set(poleMafTruncatingTSG['allele']) - set(recurrentTruncatingAlleles)

In [ ]:
#recurrentTruncatingAlleles
notRecurrentTruncatingAlleles

In [ ]:
listOfDicts = []

#We get averages which we will use for our ordering val
alleleFracDict = {}
for allele in significantAlleles:
    gene = allele.split('_')[0]
    fracCasesWithMultiplet = count_number_of_times_allele_is_in_multiplet(poleMafTruncatingTSG, allele)
    alleleFracDict[allele] = fracCasesWithMultiplet
    #geneClass = 'unrelated'
    #if gene in cohortRelatedGenes:
    #    geneClass = 'related'
    #listOfDicts.append({'gene': gene, 'frac': fracCasesWithMultiplet, 
    #            'allele': allele, 'geneClass': geneClass})

#ABOVE IS A DEPRECATED WAY OF DOING THINGS
                
#do a second iteration where we get summary figures for the recurrent and non recurrent alleles

"""relatedVals = []
unrelatedVals = [] 
for allele in notRecurrentTruncatingAlleles:
    gene = allele.split('_')[0]
    fracCasesWithMultiplet = count_number_of_times_allele_is_in_multiplet(poleMafTruncatingTSG, allele)
    if gene in cohortRelatedGenes:
        relatedVals.append(fracCasesWithMultiplet)
    else:
        unrelatedVals.append(fracCasesWithMultiplet)
"""

#alleleFracDict['Average other \nrelated gene alleles'] :  np.nanmean(relatedVals)
#alleleFracDict['Average other \nunrelated gene alleles'] :  np.nanmean(relatedVals)
     
#listOfDicts.append({
    #'Hugo_Symbol': 'Average other \nrelated gene alleles', 'frac': np.nanmean(relatedVals), 
 #               'allele':'Average other \nrelated gene alleles', 'geneClass': 'related_average'})
#listOfDicts.append({
    #'Hugo_Symbol': 'Average other \nunrelated gene alleles', 'frac': np.nanmean(unrelatedVals), 
    #            'allele': 'Average other \nunrelated gene alleles', 'geneClass': 'unrelated_average'})


In [ ]:
#add columns to the maf so R can plot it with error bars
poleMafTruncatingTSG['displayAllele'] = poleMafTruncatingTSG['allele'].apply(lambda x: x if x in significantAlleles
                    else 'Average other \nrelated gene alleles' if allele.split('_')[0] in cohortRelatedGenes
                    else 'Average other \nunrelated gene alleles')
poleMafTruncatingTSG['multipletPresentInCase'] = poleMafTruncatingTSG.apply(lambda row:
                is_allele_multiplet(poleMafTruncatingTSG, row['allele'], row['Tumor_Sample_Barcode']), axis=1)
poleMafTruncatingTSG['geneClass'] = poleMafTruncatingTSG['Hugo_Symbol'].apply(lambda x: 'related' if x in cohortRelatedGenes else 'unrelated')

In [ ]:
#poleMafTruncatingTSG.columns.values
df = poleMafTruncatingTSG[['Hugo_Symbol', 'allele', 'multipletPresentInCase', 'displayAllele', 'geneClass']]

In [ ]:
df['frac'] = df['allele'].apply(lambda x: alleleFracDict[x] if x in alleleFracDict else 0)

In [ ]:
#order the plot
df['orderingVal'] = df.apply(lambda row: -.01 if row['displayAllele'] == 'Average other \nrelated gene alleles'
                             else 1.01 if row['displayAllele'] == 'Average other \nunrelated gene alleles'
                            else 1- row['frac'] if row['geneClass'] == 'related'
                            else 2.01 -row['frac'], axis=1)


In [ ]:
df.to_csv('/Users/friedman/Desktop/WORK/dataForLocalPlotting/stopGainAlleleRecurrenceAndDoubles.tsv', index=False, sep='\t')

In [ ]:
poleMafTruncatingTSG[poleMafTruncatingTSG['allele']== 'APC_p.R2204*']['geneClass']

**HOTSPOT mutations never seen**

In [ ]:
def assign_hotspot_freq_dict(df):
    d = {}
    for index, row in df.iterrows():
        
        refAminoAcid = row['ref']
        gene = row['Hugo_Symbol']
        position = row['Amino_Acid_Position']
        for entry in row['Var_AA'].split('|'):
            fullAltName = ''
            altAminoAcid, count = entry.split(':')
            fullAltName = gene + ':' + refAminoAcid + position + altAminoAcid
            d[fullAltName] = float(count)
    return d


In [ ]:
hotspotsNeverSeen = set(df[df['count'] < 1]['allele'])

In [ ]:
hotspotsDf = pd.read_table(pathPrefix + '/home/gavrilae/snp_output_final_pancan.txt')
hotspotIncidenceD = assign_hotspot_freq_dict(hotspotsDf)

In [ ]:
listOfDicts = []
for hotspot in hotspotsNeverSeen:
    adjHotspot = re.sub('_p.', ':', hotspot)
    if adjHotspot in hotspotIncidenceD:
        listOfDicts.append({'allele': adjHotspot, 'count': hotspotIncidenceD[adjHotspot]})
    else:
        listOfDicts.append({'allele': adjHotspot, 'count': 0})
df = pd.DataFrame(listOfDicts)

In [ ]:
thresh = 10
df['alleleLabel'] = df.apply(lambda row: row['allele'] if row['count'] > thresh else None, axis=1)

In [ ]:
df.to_csv('/Users/friedman/Desktop/WORK/neverSeenHotspot24kCounts.tsv', index=False, sep='\t')

**analysis of palindromic pole susceptible signature**

In [ ]:
palindromicDf = pd.read_table(pathPrefix + '/ifs/work/taylorlab/friedman/myAdjustedDataFiles/palindromicPoleContextSummary.tsv')


In [ ]:
palindromicDf['genePosition'] = palindromicDf.apply(lambda row: row['Hugo_Symbol'] + '_' + str(row['aminoAcidPosition']), axis=1)

In [ ]:
listOfDicts = []
for position in set(palindromicDf['genePosition']):
    positionDf = palindromicDf[palindromicDf['genePosition'] == position]
    alleles = set(positionDf['HGVSp_Short'])
    gene = positionDf['Hugo_Symbol'].iloc[0]
    
    rqVal = 0
    rStarVal = 0
    for allele in alleles:
        nmut = poleMafWithPentaContext[(poleMafWithPentaContext['Hugo_Symbol'] == gene) & (poleMafWithPentaContext['HGVSp_Short'] == allele)].shape[0]
        if allele[len(allele) - 1] == '*':
            rStarVal = nmut
        else:
            rqVal = nmut
    dif = rStarVal - rqVal 
    total = rStarVal + rqVal
    listOfDicts.append({'gene': gene, 'position': position, 'residueName': str(position),
                       'StarMinusQ': dif, 'total': total})
    
df = pd.DataFrame(listOfDicts)
    
    
    

In [ ]:
thresh = 5
df['displayName'] = df.apply(lambda row: row['residueName'] if abs(row['StarMinusQ']) >= thresh else None, axis=1)

In [ ]:
#assign counts for N alleles at each coordinate 
df['coord'] = df.apply(lambda row: str(row['StarMinusQ']) + ',' + str(row['total']), axis=1)
coordCounter = Counter(df['coord'])
df['nAllelesAtCoord'] = df['coord'].apply(lambda x: coordCounter[x])

In [ ]:
df.to_csv('/Users/friedman/Desktop/WORK/dataForLocalPlotting/palindromicAlleleAnalysis.tsv', index=False, sep='\t')

**Get z scores for SNP mutations**

In [59]:
def enumerate_never_seen_mutations(allPossibleMaf, cohortMaf, pentaChange):
    allMutQuadnucMaf = allPossibleMaf[allPossibleMaf['pentaChange'] == pentaChange]
    cohortQuadnucMaf = cohortMaf[cohortMaf['pentaChange'] == pentaChange]
    
    possibleAllelesAtQuadnuc = set(allMutQuadnucMaf['allele'])
    
    seenAlleles = set(cohortQuadnucMaf[cohortQuadnucMaf['allele'].isin(possibleAllelesAtQuadnuc)]['allele'])
    neverSeenAlleles = possibleAllelesAtQuadnuc - seenAlleles
    
    neverSeenSummary = allMutQuadnucMaf[allMutQuadnucMaf['allele'].isin(neverSeenAlleles)]
    seenSummary = allMutQuadnucMaf[allMutQuadnucMaf['allele'].isin(seenAlleles)]
    seenAlleleCountsDict = dict(cohortQuadnucMaf['allele'].value_counts())
    
    neverSeenSummary['alleleCount'] = 0
    seenSummary['alleleCount'] = seenSummary['allele'].apply(lambda x: seenAlleleCountsDict[x])
    
    combinedDf = pd.concat([neverSeenSummary, seenSummary])
    combinedDf['pentaChange'] = pentaChange
    finalDf = combinedDf[['pentaChange', 'alleleCount', 'allele', 'Consequence', 'is-a-hotspot', 'oncogenic']]
    return finalDf
    

In [180]:
#assigns z scores to every mutation in the maf with z scores being calucluated based on quadnuc
def assign_motif_specific_z_scores_to_mutations(maf, motifColName='pentaChange'):
    maf['zScore'] = None
    maf['motifMean'] = None
    motifs = set(maf[motifColName])
    for m in motifs:
        print m
        motifMaf = maf[maf[motifColName] == m]
        alleles = motifMaf['allele']
        counts = motifMaf['alleleCount']
        transformedCounts = motifMaf['transformedCount']
        meanTCount = np.nanmean(transformedCounts)
        
        zscores = stats.zscore(transformedCounts)
        mapping = dict(zip(alleles, zscores))
        maf['zScore'] = maf.apply(lambda row: mapping[row['allele']] if row['allele'] in mapping else row['zScore'], axis=1)
        maf['motifMean'] = maf.apply(lambda row: meanTCount if row[motifColName] == m else row['motifMean'], axis=1)
    return maf


In [ ]:
dfAllPossibleAlleles = analysis_utils.load_in_df_with_progress(filePath = pathPrefix + '/ifs/work/taylorlab/friedman/myAdjustedDataFiles/allMutsContextSummary.tsv', nLinesFile = 4243342)



In [58]:
dfAllPossibleAlleles['pentaChange'] = dfAllPossibleAlleles.apply(lambda row: 
                                    mutationSigUtils.create_strand_specific_pentanucleotide_change(row['Ref_Tri.1'], row['Reference_Allele'], row['Tumor_Seq_Allele2'], row['Variant_Type']), axis=1)


In [60]:
poleMafWithPentaContext = pd.read_table(pathPrefix + '/ifs/work/taylorlab/friedman/myAdjustedDataFiles/poleCaseMafWithPentanucleotideContext.maf')
poleMafWithPentaContext['pentaChange'] = poleMafWithPentaContext.apply(lambda row: 
                                    mutationSigUtils.create_strand_specific_pentanucleotide_change(row['Ref_Tri.1'], row['Reference_Allele'], row['Tumor_Seq_Allele2'], row['Variant_Type']), axis=1)


In [ ]:
listOfDfs = []
for penta in topTenPolePentas:
    listOfDfs.append(enumerate_never_seen_mutations(dfAllPossibleAlleles, poleMafWithPentaContext, penta))
combinedDf = pd.concat(listOfDfs)

In [ ]:
combinedDfLimited = combinedDf[combinedDf['Consequence'].isin(
    ['missense_variant', 'missense_variant;splice_region_variant', 'stop_gained'])]
combinedDfLimited['mutationType'] = combinedDfLimited.apply(lambda row: 'hotspot' if row['is-a-hotspot'] == 'Y'
        else 'oncogenic truncating' if (row['oncogenic'] == 'Likely Oncogenic') and (row['Consequence'] == 'stop_gained')
        else 'other oncogenic' if row['oncogenic'] == 'Likely Oncogenic'                                                     
        else 'other truncating' if row['Consequence'] == 'stop_gained' else 'other non-synonymous', axis=1)

In [ ]:
combinedDfLimited['transformedCount'] = combinedDfLimited['alleleCount'].apply(lambda x: 2*math.sqrt(x))

In [ ]:
combinedDfLimitedWithZScores = assign_motif_specific_z_scores_to_mutations(combinedDfLimited)

In [125]:
combinedDfLimitedWithZScores['displayLabel'] = combinedDfLimitedWithZScores.apply(lambda row:
        row['allele'] if row['transformedCount'] > row['pentaNucMean'] + 4 else None, axis=1)

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [126]:
combinedDfLimitedWithZScores.to_csv('/Users/friedman/Desktop/WORK/dataForLocalPlotting/pentaPOLECounts.tsv', index=False, sep='\t')

**Summarize info for MMR tumors**

In [71]:
reload(msi_analysis_utils)
msiSummary = pd.read_table('/Users/friedman/Desktop/impact_microsatellites.txt')


/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (0,76,97) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [72]:
allImpactMuts = pd.read_table('/Users/friedman/Desktop/offlineFilesForVirus/data_mutations_extended_annotated_sigContext_nov19_2019.maf')
#allImpactMuts = analysis_utils.load_in_df_with_progress(filePath = pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/all_impact_mutations_annotated_cohort.maf', nLinesFile = 275000)
allImpactMuts['allele'] = allImpactMuts.apply(lambda row: str(row['Hugo_Symbol']) + '_' + str(row['HGVSp_Short']), axis=1)
msiSummary['allele'] = msiSummary.apply(lambda row: str(row['Hugo_Symbol']) + '_' + str(row['HGVSp_Short']), axis=1)

/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (4,17,18,40,41,43,44,71,72,108,124,125,128,129,130,131,132,133,134,135,137,140,141) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [73]:
neverObservedSites, msiSitesToNameMapping, mafMsiSiteToNameMapping =  msi_analysis_utils.standardize_allele_names(msiSummary, allImpactMuts)

5.74382538771 percent done
11.4876507754 percent done
17.2314761631 percent done
22.9753015508 percent done
28.7191269385 percent done
34.4629523262 percent done
40.206777714 percent done
45.9506031017 percent done
51.6944284894 percent done
57.4382538771 percent done
63.1820792648 percent done
68.9259046525 percent done
74.6697300402 percent done
80.4135554279 percent done
86.1573808156 percent done
91.9012062033 percent done
97.645031591 percent done


In [75]:
allImpactMuts['correctedAllele'] = allImpactMuts['allele'].apply(lambda x:
                                                                 mafMsiSiteToNameMapping[x] if x in mafMsiSiteToNameMapping else None)
msiSummary['correctedAllele'] = msiSummary['allele'].apply(lambda x:
                                                           msiSitesToNameMapping[x] if x in msiSitesToNameMapping else None)


In [76]:
allImpactMuts = maf_analysis_utils.fix_mll_genes(allImpactMuts)

In [78]:
msiCases = get_gene_and_cohort_list_utils.get_msi_cases(msiInfoFilePath = pathPrefix + '/juno/work/taylorlab/friedman/myAdjustedDataFiles/mutations_TMB_and_MSI_stats.txt', msiScoreThresh=10)
msiMaf = allImpactMuts[allImpactMuts['Tumor_Sample_Barcode'].isin(msiCases)]

In [80]:
reload(msi_analysis_utils)
occurenceSummaryDf = msi_analysis_utils.summarize_mutation_counts_and_msi_allele_relationship(msiMaf, msiSummary)

In [82]:
occurenceSummaryDf['basePairClass'] = occurenceSummaryDf.apply(lambda row: str(row['repeat_length']) + '_AT' if row['basePair'] in set(['A', 'T'])
                else str(row['repeat_length']) + '_CG' if row['basePair'] in set(['C', 'G']) else str(row['repeat_length']) + '_other', axis=1)


In [197]:
occurenceSummaryDf['alleleCount'] = occurenceSummaryDf['nOccurences']
occurenceSummaryDf['transformedCount'] = occurenceSummaryDf['alleleCount'].apply(lambda x: 2*math.sqrt(x))
combinedMsiSummaryDf = assign_motif_specific_z_scores_to_mutations(occurenceSummaryDf, motifColName='basePairClass')
 

8_other
22_other
7_CG
6_other
3_other
10_AT
5_CG
8_CG
9_other
5_other
6_AT
5_AT
4_other
7_other
10_other
9_AT
8_AT
7_AT
6_CG


In [198]:
combinedMsiSummaryDf['displayLabel'] = combinedMsiSummaryDf.apply(lambda row:
        row['allele'] if row['transformedCount'] > row['motifMean'] + 4 else None, axis=1)

tumorSuppresors = set(['ERRFI1', 'ASXL2', 'PMAIP1', 'ACTG1', 'SUFU', 'FBXO11', 'MEN1', 'FAM58A', 'B2M', 'RB1', 'DUSP22', 'SESN1', 'GPS2', 'RAD51D', 'SMG1', 'CDC73', 'MAP3K1', 'SMARCB1', 'INPP4B', 'PARK2', 'SMAD4', 'CBFB', 'CDH1', 'PPP6C', 'SETDB1', 'SETDB2', 'NF2', 'CDKN2B', 'CDKN2C', 'CDKN2A', 'DDX3X', 'PIK3R1', 'BARD1', 'PDS5B', 'KLF4', 'SPRED1', 'VHL', 'SMAD2', 'PMS1', 'PMS2', 'SETD2', 'GATA3', 'TBL1XR1', 'MUTYH', 'SOCS1', 'FAM175A', 'ROBO1', 'ARID1B', 'ARID1A', 'TCF7L2', 'STK11', 'FOXA1', 'PTEN', 'FAT1', 'FAS', 'CYLD', 'MAX', 'SH2D1A', 'APC', 'NTHL1', 'CTCF', 'KDM5C', 'KMT2C', 'ZFHX3', 'FOXP1', 'PIGA', 'CDKN1B', 'CDKN1A', 'FUBP1', 'MSH2', 'ID3', 'TNFRSF14', 'TRAF3', 'EP400', 'BRIP1', 'ARID4A', 'ARID4B', 'XRCC2', 'DAXX', 'SDHAF2', 'ASXL1', 'AMER1', 'RASA1', 'EGR1', 'MST1', 'SOX17', 'RUNX1', 'PIK3R3', 'NCOR1', 'NF1', 'JAK1', 'PTPRD', 'CHEK2', 'CHEK1', 'SMC1A', 'TMEM127', 'STAG1', 'RAD51', 'TCF3', 'STAG2', 'ARID2', 'RAD50', 'RNF43', 'PARP1', 'BLM', 'CUX1', 'RECQL', 'RAD21', 'PTPN2', 'PTPN1', 'SLX4', 'INHA', 'PAX5', 'IRF1', 'TP53', 'HLA-A', 'IRF8', 'CBL', 'TOP1', 'SHQ1', 'PRDM1', 'NSD1', 'ATXN2', 'CREBBP', 'HDAC4', 'SESN2', 'PPP2R1A', 'EPHA7', 'ATM', 'EPHA3', 'POT1', 'SMAD3', 'MOB3B', 'TBX3', 'POLE', 'ATR', 'FANCD2', 'FH', 'BCORL1', 'SOX9', 'IKZF3', 'TSC1', 'TP63', 'MRE11A', 'SDHC', 'BTG1', 'POLD1', 'CIITA', 'SMC3', 'SAMHD1', 'RTEL1', 'ECT2L', 'PIK3R2', 'CRBN', 'FANCC', 'NBN', 'FANCA', 'HLA-B', 'RECQL4', 'DUSP4', 'ERCC2', 'FBXW7', 'TGFBR2', 'TGFBR1', 'MSH3', 'RBM15', 'TET1', 'TET3', 'SESN3', 'MGA', 'LTB', 'FOXL2', 'SH2B3', 'BCOR', 'HIST1H1D', 'ATRX', 'EP300', 'RAD51C', 'RAD51B', 'HIST1H1B', 'TNFAIP3', 'DICER1', 'ARID5B', 'LATS2', 'FOXO1', 'KEAP1', 'EZH2', 'SP140', 'NKX3-1', 'PBRM1', 'PALB2', 'CIC', 'BRCA1', 'DTX1', 'FLCN', 'SPEN', 'CD58', 'ERCC3', 'ERCC4', 'MSH6', 'BCL11B', 'BMPR1A', 'ERF', 'BRCA2', 'NOTCH2', 'EED', 'MITF', 'ELF3', 'SMARCA4', 'BBC3', 'ANKRD11', 'CEBPA', 'BCL2L11', 'AXIN2', 'AXIN1', 'CDK12', 'ESCO2', 'MLH1', 'SDHB', 'MED12', 'HNF1A', 'RYBP', 'ATP6V1B2', 'DNMT3B', 'KMT2B', 'KMT2A', 'DNMT3A', 'NFKBIA', 'TRAF5', 'KMT2D', 'SPOP', 'RBM10', 'P2RY8', 'TP53BP1', 'TSC2', 'KDM6A', 'EPCAM', 'PHOX2B', 'NPM1', 'BCL10', 'LATS1', 'HOXB13', 'ARID3A', 'PTPRT', 'PTPRS', 'INPPL1', 'NOTCH4', 'TET2', 'NOTCH1', 'CASP8', 'NOTCH3', 'GRIN2A', 'MAP2K4', 'WT1', 'BACH2', 'SDHA', 'BAP1', 'PTCH1', 'SDHD'])
combinedMsiSummaryDf['geneType'] = combinedMsiSummaryDf['Hugo_Symbol'].apply(lambda x: 'TSG' if x in tumorSuppresors else 'Oncogene')


In [199]:
combinedMsiSummaryDf.to_csv('/Users/friedman/Desktop/WORK/dataForLocalPlotting/msiMotifCounts.tsv', index=False, sep='\t')

**Make chart for recurrent alleles**

In [215]:
def summarize_msi_allele_data_for_plotting(df):
    totalN = sum(df['nOccurences'])
    tsgDf = df[df['Hugo_Symbol'].isin(tumorSuppresors)]
    oncogeneDf = df[~df['Hugo_Symbol'].isin(tumorSuppresors)]
    listOfDs = [{'fracClass': (1.0*sum(tsgDf['nOccurences']))/totalN , 'geneType': 'TSG', 'class': 'MSI INDEL'},
                {'fracClass': (1.0*sum(oncogeneDf['nOccurences']))/totalN, 'geneType': 'Oncogene', 'class': 'MSI INDEL'},
               ]
    return pd.DataFrame(listOfDs)

def summarize_pole_allele_data_for_plotting(df):
    totalN = sum(df['alleleCount'])
    tsgDf = df[df['Hugo_Symbol'].isin(tumorSuppresors)]
    oncogeneDf = df[~df['Hugo_Symbol'].isin(tumorSuppresors)]
    listOfDs = [{'fracClass': (1.0*sum(tsgDf['alleleCount']))/totalN , 'geneType': 'TSG', 'class': 'POLE STOP-GAIN'},
                {'fracClass': (1.0*sum(oncogeneDf['alleleCount']))/totalN, 'geneType': 'Oncogene', 'class': 'POLE STOP-GAIN'},
               ]
    return pd.DataFrame(listOfDs)
    

In [210]:
#MSI
recurrentAllelesSummaryMsi = combinedMsiSummaryDf[combinedMsiSummaryDf['displayLabel'].notnull()]
#POLE
recurrentAllelesTruncSummaryPole = combinedDfLimitedWithZScores[(combinedDfLimitedWithZScores['displayLabel'].notnull()) & (combinedDfLimitedWithZScores['Consequence'] == 'stop_gained')]
recurrentAllelesTruncSummaryPole['Hugo_Symbol'] = recurrentAllelesTruncSummaryPole['allele'].apply(lambda x: x.split('_')[0])

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [216]:
msiDf = summarize_msi_allele_data_for_plotting(recurrentAllelesSummaryMsi)
poleDf = summarize_pole_allele_data_for_plotting(recurrentAllelesTruncSummaryPole)

In [217]:
combinedDf = pd.concat([msiDf, poleDf])
combinedDf.to_csv('/Users/friedman/Desktop/WORK/dataForLocalPlotting/tsgFracsRecurrentAllelesPOLE_MMR.tsv', index=False, sep='\t')

In [ ]:
#
#####
#############
####################
############################
######################
##############
######
#

#SUMMARIZE OUTLIER

In [119]:
def summarize_outlier_scores_across_alleles(maf, contexts, contextCol):
    listOfDicts = []
    for context in contexts:
        contextMaf = maf[maf[contextCol] == context]
        alleleCounts = maf['allele'].value_counts()
        for allele in set(contextMaf['allele']):
            count = alleleCounts[allele]
            gene = allele.split('_')[0]
            listOfDicts.append({'allele': allele, 'count': count,
                                'context': context, 'gene': gene})
    df = pd.DataFrame(listOfDicts)
    return df

#for now we have a separate summary function for msi given the formatting I did in msiUtils
def summarize_outlier_scores_across_alleles_msi(sumDf, contexts):
    listOfDicts = []
    for context in contexts:
        contextSummaryDf = sumDf[sumDf['basePairClass'] == context]
        nOccurencesDict = dict(zip(contextSummaryDf['allele'], contextSummaryDf['nOccurences']))
        for allele, count in nOccurencesDict.items():
            gene = allele.split('_')[0]
            listOfDicts.append({'allele': allele, 'count': count,
                                'context': context, 'gene': gene})
    df = pd.DataFrame(listOfDicts)
    return df

In [120]:
#NOTE WE NEED TO USE THE ABOVE CODE LOGIC TO PROPELY SET UP THE POLE AND MSI MAFs
pentaContexts = dict(Counter(poleMafWithPentaContext['pentaChange']).most_common(10)).keys()
dfPole = summarize_outlier_scores_across_alleles(poleMafWithPentaContext, pentaContexts, 'pentaChange')

msiContexts = dict(Counter(occurenceSummaryDf['basePairClass']).most_common(10)).keys()
dfMsi = summarize_outlier_scores_across_alleles_msi(occurenceSummaryDf, msiContexts)


In [121]:
dfPole['signatureClass'] = 'POLE'
dfMsi['signatureClass'] = 'MSI'
df = pd.concat([dfPole, dfMsi])

In [135]:
df['adjCount'] = df['count'].apply(lambda x: 2*math.sqrt(x))
tumorSuppresors = set(['ERRFI1', 'ASXL2', 'PMAIP1', 'ACTG1', 'SUFU', 'FBXO11', 'MEN1', 'FAM58A', 'B2M', 'RB1', 'DUSP22', 'SESN1', 'GPS2', 'RAD51D', 'SMG1', 'CDC73', 'MAP3K1', 'SMARCB1', 'INPP4B', 'PARK2', 'SMAD4', 'CBFB', 'CDH1', 'PPP6C', 'SETDB1', 'SETDB2', 'NF2', 'CDKN2B', 'CDKN2C', 'CDKN2A', 'DDX3X', 'PIK3R1', 'BARD1', 'PDS5B', 'KLF4', 'SPRED1', 'VHL', 'SMAD2', 'PMS1', 'PMS2', 'SETD2', 'GATA3', 'TBL1XR1', 'MUTYH', 'SOCS1', 'FAM175A', 'ROBO1', 'ARID1B', 'ARID1A', 'TCF7L2', 'STK11', 'FOXA1', 'PTEN', 'FAT1', 'FAS', 'CYLD', 'MAX', 'SH2D1A', 'APC', 'NTHL1', 'CTCF', 'KDM5C', 'KMT2C', 'ZFHX3', 'FOXP1', 'PIGA', 'CDKN1B', 'CDKN1A', 'FUBP1', 'MSH2', 'ID3', 'TNFRSF14', 'TRAF3', 'EP400', 'BRIP1', 'ARID4A', 'ARID4B', 'XRCC2', 'DAXX', 'SDHAF2', 'ASXL1', 'AMER1', 'RASA1', 'EGR1', 'MST1', 'SOX17', 'RUNX1', 'PIK3R3', 'NCOR1', 'NF1', 'JAK1', 'PTPRD', 'CHEK2', 'CHEK1', 'SMC1A', 'TMEM127', 'STAG1', 'RAD51', 'TCF3', 'STAG2', 'ARID2', 'RAD50', 'RNF43', 'PARP1', 'BLM', 'CUX1', 'RECQL', 'RAD21', 'PTPN2', 'PTPN1', 'SLX4', 'INHA', 'PAX5', 'IRF1', 'TP53', 'HLA-A', 'IRF8', 'CBL', 'TOP1', 'SHQ1', 'PRDM1', 'NSD1', 'ATXN2', 'CREBBP', 'HDAC4', 'SESN2', 'PPP2R1A', 'EPHA7', 'ATM', 'EPHA3', 'POT1', 'SMAD3', 'MOB3B', 'TBX3', 'POLE', 'ATR', 'FANCD2', 'FH', 'BCORL1', 'SOX9', 'IKZF3', 'TSC1', 'TP63', 'MRE11A', 'SDHC', 'BTG1', 'POLD1', 'CIITA', 'SMC3', 'SAMHD1', 'RTEL1', 'ECT2L', 'PIK3R2', 'CRBN', 'FANCC', 'NBN', 'FANCA', 'HLA-B', 'RECQL4', 'DUSP4', 'ERCC2', 'FBXW7', 'TGFBR2', 'TGFBR1', 'MSH3', 'RBM15', 'TET1', 'TET3', 'SESN3', 'MGA', 'LTB', 'FOXL2', 'SH2B3', 'BCOR', 'HIST1H1D', 'ATRX', 'EP300', 'RAD51C', 'RAD51B', 'HIST1H1B', 'TNFAIP3', 'DICER1', 'ARID5B', 'LATS2', 'FOXO1', 'KEAP1', 'EZH2', 'SP140', 'NKX3-1', 'PBRM1', 'PALB2', 'CIC', 'BRCA1', 'DTX1', 'FLCN', 'SPEN', 'CD58', 'ERCC3', 'ERCC4', 'MSH6', 'BCL11B', 'BMPR1A', 'ERF', 'BRCA2', 'NOTCH2', 'EED', 'MITF', 'ELF3', 'SMARCA4', 'BBC3', 'ANKRD11', 'CEBPA', 'BCL2L11', 'AXIN2', 'AXIN1', 'CDK12', 'ESCO2', 'MLH1', 'SDHB', 'MED12', 'HNF1A', 'RYBP', 'ATP6V1B2', 'DNMT3B', 'KMT2B', 'KMT2A', 'DNMT3A', 'NFKBIA', 'TRAF5', 'KMT2D', 'SPOP', 'RBM10', 'P2RY8', 'TP53BP1', 'TSC2', 'KDM6A', 'EPCAM', 'PHOX2B', 'NPM1', 'BCL10', 'LATS1', 'HOXB13', 'ARID3A', 'PTPRT', 'PTPRS', 'INPPL1', 'NOTCH4', 'TET2', 'NOTCH1', 'CASP8', 'NOTCH3', 'GRIN2A', 'MAP2K4', 'WT1', 'BACH2', 'SDHA', 'BAP1', 'PTCH1', 'SDHD'])
df['geneType'] = df['gene'].apply(lambda x: 'TSG' if x in tumorSuppresors else 'Oncogene')

hotspots = set(poleMafWithPentaContext[poleMafWithPentaContext['is-a-hotspot'] == 'Y']['allele'])
oncogenic = set(poleMafWithPentaContext[poleMafWithPentaContext['oncogenic'].notnull()]['allele'])
msiIndels = set(dfMsi['allele'])
df['mutationType'] = df.apply(lambda row:
        'Oncogene_Hotspot' if (row['allele'] in hotspots and row['geneType'] == 'Oncogene')
        else 'TSG_hotspot' if (row['allele'] in hotspots and row['geneType'] == 'TSG')
        else 'oncogene_oncogenic' if (row['allele'] in oncogenic and row['geneType'] == 'Oncogene')
        else 'TSG_oncogenic' if (row['allele'] in oncogenic and row['geneType'] == 'TSG')
        else 'TSG_msi_indel' if (row['allele'] in msiIndels and row['geneType'] == 'TSG')
        else 'Oncogene_msi_indel' if (row['allele'] in msiIndels and row['geneType'] == 'Oncogene')
        else 'VUS', axis=1)

contextMeanDict = {}
for context in set(df['context']):
    contextDf = df[df['context'] == context]
    mean = np.nanmean(contextDf['adjCount'])
    contextMeanDict[context] = mean
df['meanDif'] = df.apply(lambda row: row['adjCount'] - contextMeanDict[row['context']], axis=1)
    

In [138]:
df.to_csv('/Users/friedman/Desktop/WORK/dataForLocalPlotting/recurrentAlleleCounts.tsv', index=False, sep='\t')

**EXOME**

In [7]:
#TODO there is an error and trinucleotides didnt add properly
exomeHypermutatorMaf = pd.read_table(filePathDict['ALL_EXOME_HYPERMUTATOR_MAF_WITH_TRINUC'])


/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  


In [8]:
tcgaMmrIds, tcgaPoleIds = get_gene_and_cohort_list_utils.get_tcga_pole_mmr_hypermutator_ids(
    tcgaSigsPath = filePathDict['TCGA_SIGNATURE_DECOMPOSITIONS']) 
recaptureMmrIds, recapturePoleIds = get_gene_and_cohort_list_utils.get_exome_recapture_pole_mmr_hypermutator_ids(exomeRecaptureSigsPath = filePathDict['EXOME_RECAPTURE_SIGNATURE_DECOMPOSITIONS'])
poleHypermutatorMaf = exomeHypermutatorMaf[exomeHypermutatorMaf['Tumor_Sample_Barcode'].isin(tcgaPoleIds | recapturePoleIds)]
mmrHypermutatorMaf = exomeHypermutatorMaf[exomeHypermutatorMaf['Tumor_Sample_Barcode'].isin(tcgaMmrIds | recaptureMmrIds)]


In [11]:
essentialGenes = get_gene_and_cohort_list_utils.get_essential_genes(depMapPath = filePathDict['DEP_MAP_DATA'], mode='getEssentialGenes')
tsgs = get_gene_and_cohort_list_utils.get_tsgs()
oncogenes = get_gene_and_cohort_list_utils.get_oncogenes()


In [26]:
depMapData = get_gene_and_cohort_list_utils.get_essential_genes(depMapPath = filePathDict['DEP_MAP_DATA'], mode='getAllGenes')

In [28]:
superEssentialGenes = set(depMapData[depMapData['score'] < -1.5]['Hugo_Symbol'])

In [31]:
#print Counter(mmrHypermutatorMaf['Variant_Classification']).most_common(20)
truncatingConsequences = ['Frame_Shift_Ins', 'Frame_Shift_Del', 'Nonsense_Mutation']
print np.nanmean(mmrHypermutatorMaf[
    mmrHypermutatorMaf['Hugo_Symbol'].isin(tsgs) & 
    (mmrHypermutatorMaf['Variant_Classification'].isin(truncatingConsequences))]['ccf_Mcopies'])

0.8037213032581454


In [ ]:
codingVariantClasses = set(['Frame_Shift_Del', 'Frame_Shift_Ins', 'In_Frame_Del', 'In_Frame_Ins',
                           'Missense_Mutation', 'Nonsense_Mutation', 'Nonstop_Mutation', 'Splice_Site'])
poleHypermutatorMaf['varUuid'] = poleHypermutatorMaf.apply(lambda row:
    str(row['Hugo_Symbol']) + '_' + str(row['HGVSp_Short']) if row['Variant_Classification'] in codingVariantClasses
    else str(row['Hugo_Symbol']) + '_' + str(row['Start_Position']), axis=1)
mmrHypermutatorMaf['varUuid'] = mmrHypermutatorMaf.apply(lambda row:
    str(row['Hugo_Symbol']) + '_' + str(row['HGVSp_Short']) if row['Variant_Classification'] in codingVariantClasses
    else str(row['Hugo_Symbol']) + '_' + str(row['Start_Position']), axis=1)

In [5]:
for allele, count in Counter(mmrHypermutatorMaf['varUuid']).most_common(10):
    print allele, count,
    print mmrHypermutatorMaf[mmrHypermutatorMaf['varUuid'] == allele]['Chromosome'].iloc[0],
    print mmrHypermutatorMaf[mmrHypermutatorMaf['varUuid'] == allele]['Start_Position'].iloc[0]
    
    

RPL22_p.K15Rfs*5 84 1 6257785
RNF43_p.G659Vfs*41 81 17 56435161
DOCK3_p.P1852Qfs*45 74 3 51417604
ACVR2A_p.K437Rfs*5 70 2 148683686
DRD5_9785349 61 4 9785349
ZBTB20_p.P692Lfs*43 59 3 114058003
LARP4B_p.T163Hfs*47 45 10 890939
PLEKHA6_p.V328Yfs*172 41 1 204228411
PGM5_p.I98V 39 9 70993145
ELMSAN1_p.N314Tfs*4 37 14 74205773


In [31]:
print Counter(exomeHypermutatorMaf[exomeHypermutatorMaf['Ref_Tri'].notnull()]['cohort'])
#Counter(poleHypermutatorMaf['Ref_Tri'])
#for trinuc in Counter(poleHypermutatorMaf['Ref_Tri']).most_common(10):
#    print trinuc
    

Counter({'mmr_RECAPTURE': 113257, 'pole_RECAPTURE': 3175})
